In [2]:
import pickle

saved_model = open('models/tweet_model.pickle', 'rb')

loaded_classifier = pickle.load(saved_model)

saved_model.close()

In [3]:
import re
from collections import Counter

import nltk.classify.util
import pandas as pd
from nltk.classify import NaiveBayesClassifier
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import TweetTokenizer

POSITIVE_COMMENTS_CSV = 'datasets/clean_comments_neg.csv'
NEGATIVE_COMMENTS_CSV = 'datasets/clean_comments_pos.csv'

VOCAB_SIZE = 5000

tweets_col_number = 6

negative_comments = pd.read_csv(
    POSITIVE_COMMENTS_CSV, header=None, dialect='excel-tab')[[tweets_col_number]]
positive_comments = pd.read_csv(
    NEGATIVE_COMMENTS_CSV, header=None, dialect='excel-tab')[[tweets_col_number]]


ImportError: Missing required dependencies ['numpy']

In [4]:
stemer = RussianStemmer()
regex = re.compile('[^а-яА-Я ]')
stem_cache = {}


def get_stem(token):
    stem = stem_cache.get(token, None)
    if stem:
        return stem
    token = regex.sub('', token).lower()
    stem = stemer.stem(token)
    stem_cache[token] = stem
    return stem


stem_count = Counter()
tokenizer = TweetTokenizer()

NameError: name 'RussianStemmer' is not defined

In [5]:
def count_unique_tokens_in_tweets(tweets):
    for _, tweet_series in tweets.iterrows():
        tweet = tweet_series[6]
        tokens = tokenizer.tokenize(tweet)
        for token in tokens:
            stem = get_stem(token)
            stem_count[stem] += 1


count_unique_tokens_in_tweets(negative_comments)

NameError: name 'negative_comments' is not defined

In [6]:
count_unique_tokens_in_tweets(positive_comments)

NameError: name 'positive_comments' is not defined

In [7]:
vocab = sorted(stem_count, key=stem_count.get, reverse=True)[:VOCAB_SIZE]
print(vocab[:100])

NameError: name 'stem_count' is not defined

In [8]:
def comment_to_feachure(tweet, show_unknowns=False):
    vector = []
    for token in tokenizer.tokenize(tweet):
        stem = get_stem(token)        
        if stem:
            vector.append(stem)
        elif show_unknowns:
            print("Unknown token: {}".format(token))
    return dict([('contains-word(%s)' % w, True) for w in vector])


tweet = negative_comments.iloc[1][6]
print("comment: {}".format(tweet))
print("comment_to_feachure: {}".format(comment_to_feachure(tweet)))
print(vocab[5])

NameError: name 'negative_comments' is not defined

In [9]:
comments = []

neg_comments = []
pos_comments = []

pos = 6
for ii, (_, tweet) in enumerate(negative_comments.iterrows()):
    comments.append(tweet[pos])
    neg_comments.append((comment_to_feachure(tweet[pos]), 'neg'))
for ii, (_, tweet) in enumerate(positive_comments.iterrows()):
    comments.append(tweet[pos])
    pos_comments.append((comment_to_feachure(tweet[pos]), 'pos'))

NameError: name 'negative_comments' is not defined

In [10]:
print(len(neg_comments))
print(len(pos_comments))

0
0


In [11]:
negcutoff = len(neg_comments) * 3 / 4
poscutoff = len(pos_comments) * 3 / 4

In [12]:
trainfeats = neg_comments[:int(negcutoff)] + pos_comments[:int(poscutoff)]
testfeats = neg_comments[int(negcutoff):] + pos_comments[int(poscutoff):]

In [13]:
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

train on 0 instances, test on 0 instances


In [14]:
comments_classifier = NaiveBayesClassifier.train(trainfeats)

NameError: name 'NaiveBayesClassifier' is not defined

In [15]:
print('accuracy:', nltk.classify.util.accuracy(comments_classifier, testfeats))
comments_classifier.show_most_informative_features()

NameError: name 'comments_classifier' is not defined

In [16]:
from nltk.classify import SklearnClassifier
from sklearn.tree import DecisionTreeClassifier

dts_classifier = SklearnClassifier(DecisionTreeClassifier()).train(trainfeats)

print('accuracy:', nltk.classify.util.accuracy(dts_classifier, testfeats))

ImportError: Something is wrong with the numpy installation. While importing we detected an older version of numpy in ['D:\\IDEA_projects\\StepicCommentsAnalizer\\venv\\lib\\site-packages\\numpy']. One method of fixing this is to repeatedly uninstall numpy until none is found, then reinstall this version.

In [17]:
from nltk.classify import SklearnClassifier
from sklearn.svm import SVC

svm_classifier = SklearnClassifier(SVC(gamma='auto')).train(trainfeats)


print('accuracy:', nltk.classify.util.accuracy(svm_classifier, testfeats))

ImportError: Something is wrong with the numpy installation. While importing we detected an older version of numpy in ['D:\\IDEA_projects\\StepicCommentsAnalizer\\venv\\lib\\site-packages\\numpy']. One method of fixing this is to repeatedly uninstall numpy until none is found, then reinstall this version.

In [18]:
print('accuracy:', nltk.classify.util.accuracy(loaded_classifier, testfeats))
nltk.classify.util.f1
loaded_classifier.show_most_informative_features()

accuracy: 0


AttributeError: module 'nltk.classify.util' has no attribute 'f1'

In [19]:
print('accuracy:', nltk.classify.util.accuracy(loaded_classifier, testfeats))
loaded_classifier.show_most_informative_features()

accuracy: 0
Most Informative Features
  contains-word(царевич) = True              pos : neg    =     78.9 : 1.0
  contains-word(шумахер) = True              neg : pos    =     35.2 : 1.0
 contains-word(калашник) = True              neg : pos    =     28.4 : 1.0
   contains-word(погибл) = True              neg : pos    =     28.4 : 1.0
  contains-word(позитив) = True              pos : neg    =     26.8 : 1.0
contains-word(соболезнован) = True              neg : pos    =     26.1 : 1.0
   contains-word(сконча) = True              neg : pos    =     22.2 : 1.0
   contains-word(сметан) = True              pos : neg    =     21.1 : 1.0
   contains-word(ублюдк) = True              neg : pos    =     19.5 : 1.0
  contains-word(почемуу) = True              neg : pos    =     19.5 : 1.0


In [20]:
model = open('models/comment_model.pickle', 'wb')

pickle.dump(comments_classifier,model)

model.close()

NameError: name 'comments_classifier' is not defined

In [21]:
neg_comments[1]

IndexError: list index out of range

In [22]:
comments_classifier.classify(comment_to_feachure('хорошо'))

NameError: name 'comments_classifier' is not defined

In [23]:
tokenizer_model = open('models/tokenizer_comment.pickle', 'wb')

pickle.dump(tokenizer,tokenizer_model)

tokenizer_model.close()

NameError: name 'tokenizer' is not defined

In [24]:
stem_model = open('models/stem_cache_comment.pickle', 'wb')

pickle.dump(stem_cache,stem_model)

stem_model.close()

NameError: name 'stem_cache' is not defined

In [25]:
stem_count_model = open('models/stem_count_comment.pickle', 'wb')

pickle.dump(stem_count,stem_count_model)

stem_count_model.close()

NameError: name 'stem_count' is not defined

In [26]:
model_tweet_test = open('models/tweet_model_test_features.pickle', 'rb')

test_tweet = pickle.load(model_tweet_test)

model_tweet_test.close()

print('accuracy tweet NaiveBayes:', nltk.classify.util.accuracy(loaded_classifier, test_tweet))
print('accuracy Stepic NaiveBayes:', nltk.classify.util.accuracy(comments_classifier, test_tweet))
print('accuracy svm:', nltk.classify.util.accuracy(svm_classifier, test_tweet))
print('accuracy dts:', nltk.classify.util.accuracy(dts_classifier, test_tweet))


accuracy tweet NaiveBayes: 0.7102047294080305


NameError: name 'comments_classifier' is not defined

In [27]:
svc = open('models/stem_count_comment.pickle', 'wb')

pickle.dump(svm_classifier,svc)

svc.close()

NameError: name 'svm_classifier' is not defined